In [1]:
import pandas as pd
import numpy as np
import symbol_data
import portfolio_data

In [10]:
stocks = ['AAPL', 'MSFT']

In [8]:
def stock_data_calculator(stocks):
    # Generate a dataframe of daily returns for S&P 500
    spx_daily_returns = symbol_data.get_historical_data('SPX')
    spx_daily_returns = spx_daily_returns['Close'].pct_change().dropna()
    # Create an empty dictionary to store the stock data
    stock_data_dict = {}
    # This section can be replaced by Anurag's function
    for stock in stocks:
        stock_df = symbol_data.get_historical_data(stock)
        if 'close' in stock_df.columns:
            daily_returns = stock_df['close'].pct_change().dropna()
        if 'Close' in stock_df.columns:
            daily_returns = stock_df['Close'].pct_change().dropna()
        daily_returns_df = pd.concat(
            [daily_returns, spx_daily_returns], 
            axis=1,
            join='inner', 
            keys = [stock, 'SPX']
            )
        covariance = daily_returns_df[stock].cov(daily_returns_df['SPX'])
        beta = covariance / daily_returns_df['SPX'].var()

        std = daily_returns.std()

        var = daily_returns.var()

        expected_return = .035 + beta*(.1-.035)
        
        stock_data_dict[stock] = {'expected_return': expected_return, 'std': std, 'var': var, 'beta': beta}
    return stock_data_dict

In [2]:
def portfolio_expected_return_calculator(stocks, weights):
    # Create an empty list which will hold the expected returns
    expected_returns = []
    # Gather the stock data from the symbol_data_calculator function
    stock_data = stock_data_calculator(stocks)
    # Pull only the expected returns from the data
    for stock in stock_data:
        expected_returns.append(stock_data[stock]['expected_return'])
    # Multiply the expected returns by the weights to get weighted expected returns
    weighted_expected_returns = expected_returns * weights
    # Sum all these to get the expected return of the portfolio
    weighted_expected_returns = weighted_expected_returns.sum()
    return weighted_expected_returns 

In [3]:
def portfolio_variance_calculator(stocks, weights):
    # Retrieve a dataframe of close prices for all stocks entered
    stock_df = portfolio_data.get_portfolio_historical_close_data(stocks)
    # Get the daily returns of each stock
    daily_returns = stock_df.pct_change().dropna()
    
    # Get the covariance array of the portfolio using the .cov() method.
    portfolio_cov = np.array(daily_returns.cov())

    # Get the standard deviation of each asset in the portfolio using the .std() method.
    portfolio_std = np.array(daily_returns.std())

    # Cross multiply the std array with the transposition of itself.
    portfolio_stdT = np.transpose(np.array([portfolio_std]))
    portfolio_std_cp = portfolio_std * portfolio_stdT

    # Get the correlation matrix by dividing the covariance matrix by the standard deviation matrix.
    correlation_matrix = portfolio_cov / portfolio_std_cp

    # Get weighted standard deviation and save the transposition of that array.
    weighted_std = portfolio_std * weights
    weighted_stdT = np.transpose(np.array([weighted_std]))

    # Calculate portfolio variance by first multiplying the covariance matrix by the 
    # weighted standard deviation array. This will give an array which is 
    # 1x<the number of stocks in the portfolio>
    portfolio_var = np.matmul(weighted_std, correlation_matrix)
    # Then multiplying that array by the transposition of the weighted standard deviation array.
    # This will give a single value.
    portfolio_var = np.matmul(portfolio_var, weighted_stdT)
    # And finally taking the square root of that value.
    portfolio_var = np.sqrt(portfolio_var)
    return portfolio_var

In [6]:
def efficient_frontier_generator(stocks):
    portfolio_dict = {}
    i = 0
    while i < 1000:
        weights = np.random.random(len(stocks))
        weights /= weights.sum()
        returns = portfolio_expected_return_calculator(stocks, weights)
        risk = portfolio_variance_calculator(stocks, weights)
        portfolio_dict[i] = {'returns': returns, 'risk': risk, 'weights': weights}
        i += 1
    return portfolio_dict

In [9]:
result = efficient_frontier_generator(['AAPL', 'MSFT'])

                AAPL   MSFT
               close  close
date                       
2013-02-08   67.8542  27.55
2013-02-11   68.5614  27.86
2013-02-12   66.8428  27.88
2013-02-13   66.7156  28.03
2013-02-14   66.6556  28.04
...              ...    ...
2018-02-01  167.7800  94.26
2018-02-02  160.5000  91.78
2018-02-05  156.4900  88.00
2018-02-06  163.0300  91.33
2018-02-07  159.5400  89.61

[1259 rows x 2 columns]


AttributeError: 'NoneType' object has no attribute 'pct_change'